In [11]:
!pip install segmentation_models
%env SM_FRAMEWORK=tf.keras

env: SM_FRAMEWORK=tf.keras


In [12]:
!pip install hyperdash
!hyperdash signup --github

Opening browser, please wait. If something goes wrong, press CTRL+C to cancel.
 SSH'd into a remote machine, or just don't have access to a browser? Open this link in any browser and then copy/paste the provided access token: https://hyperdash.io/oauth/github/start?state=client_cli_manual 
Waiting for Github OAuth to complete.
If something goes wrong, press CTRL+C to cancel.
Access token: AteXFJlwXQ/PovsPwUF2aDyJjm38SN3c3rP51qDddYE=
Successfully logged in! We also installed: fyX0vAjik2i9gyFJO5KWbxxBmj6KVPP+f4oqmwXMLvk= as your default API key


In [13]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/Colab Notebooks"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks


In [14]:
import numpy as np
import pandas as pd
import os
from IPython.display import display
from glob import glob
import matplotlib.pyplot as plt
import math

import segmentation_models as sm
from hyperdash import Experiment

import tensorflow as tf
from sklearn.model_selection import train_test_split

In [15]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:tensorflow:Initializing the TPU system: grpc://10.28.210.170:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.28.210.170:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [16]:
class CFG:
    debug=False

    dataset_dir="../input/ranzcr-clip-catheter-line-classification/" if debug \
        else "gs://kds-8e853601547c2d2e7a243638123e66bb32174c3a2b50da35aca5baed/"
    annotated_tfrec="../input/cvc_annotated_mask.tfrec" if debug else "gs://km_kaggle_catheter/cvc_annotated_mask.tfrec"
    tfrec_length=8853
    batch_size=2 if debug else tpu_strategy.num_replicas_in_sync*8 
    input_size=(576,576)
    epochs=1 if debug else 200

In [17]:
train_size=0.8
train_length=math.floor(CFG.tfrec_length*train_size)

In [18]:
feature_description={
    "StudyInstanceUID":tf.io.FixedLenFeature([],tf.string),
    "image":tf.io.FixedLenFeature([],tf.string)
}

def _parse_function(record):
    return tf.io.parse_example(record,feature_description)

AUTO=tf.data.experimental.AUTOTUNE

def decode_img(file_bytes):
    image=tf.io.decode_jpeg(file_bytes,channels=1)
    image=tf.image.resize(image,CFG.input_size)
    image/=255.0
    return image

def preprocess(record):
    annotated_img=decode_img(record["image"])
    uid=record["StudyInstanceUID"]
    original_path=f"{CFG.dataset_dir}train/"+uid+".jpg"
    original_img=decode_img(tf.io.read_file(original_path))
    return original_img,annotated_img

def build_dataset(train_length):
    dset=tf.data.TFRecordDataset([CFG.annotated_tfrec])
    return dset.take(train_length),dset.skip(train_length)

def dset_postprocess(dset,shuffle=256):
    dset=dset.map(_parse_function,num_parallel_calls=AUTO)
    dset=dset.map(preprocess,num_parallel_calls=AUTO)
    dset=dset.cache()
    dset=dset.shuffle(shuffle) if shuffle else dset
    dset=dset.batch(CFG.batch_size,drop_remainder=True).prefetch(AUTO)
    return dset

train_dset,val_dset=build_dataset(train_length)
train_dset=dset_postprocess(train_dset)
val_dset=dset_postprocess(val_dset)

In [22]:
load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')

def create_model():
    model=tf.keras.models.load_model("unet_ckpt",
                                     custom_objects={"iou_score":sm.metrics.IOUScore,
                                                     "binary_crossentropy_plus_jaccard_loss":sm.losses.bce_jaccard_loss},
                                     options=load_locally)
    # model=sm.PSPNet(classes=1,activation="sigmoid",encoder_weights=None,input_shape=CFG.input_size+(1,))
    # model.compile(optimizer="adam",loss=sm.losses.bce_jaccard_loss,metrics=[sm.metrics.iou_score])
    return model

if not CFG.debug:
    with tpu_strategy.scope():
        model=create_model()
else:
    model=create_model()

In [ ]:
save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
ckpt=tf.keras.callbacks.ModelCheckpoint("unet_ckpt",save_best_only=True,monitor="val_iou_score",
                                        mode="max",options=save_locally)
lr_reducer=tf.keras.callbacks.ReduceLROnPlateau(monitor="val_auc",patience=10,min_lr=1e-6,mode="max",verbose=1)

exp=Experiment("PSPNet")
model.fit(train_dset,epochs=CFG.epochs,validation_data=val_dset,callbacks=[ckpt,lr_reducer])
exp.end()

Epoch 1/200
     14/Unknown - 53s 2s/step - loss: 1.2124 - iou_score: 0.0068